<center><img src=img/MScAI_brand.png width=70%></center>

# Exceptions and Tracebacks

### Exceptions

An exception is a run-time error. It's a crash, which shows a **traceback** to help debug - unless we `catch` the exception.

Agenda:

* Common exceptions
* Reading tracebacks
* How to handle an exception with `try`-`catch`
* When and how to `raise` an exception

Various types of errors cause exceptions, and we can also  raise (or "throw") an exception in our own code. That changes
the control flow of the program. If the exception is not handled (or
"caught") inside the currently-executing function, then the function
exits -- immediately, and without returning any value. The function
that called this one then has a chance to handle (or "catch") it. If
not, it continues to the function that called *that* one, and so
on. If it is never caught, it propagates "all the way up", the program
exits ("crashes"), and you see a Traceback on your screen.

For unrecoverable errors, that is probably just fine. Other times, we
will want to handle an exception before the program exits, either
within the current function, or in an enclosing one.

So, there are two main mechanisms we need to know about: how to raise an exception, and how to handle one. We'll start by seeing lots of examples.



We've already seen quite a few exceptions. Examples:

`NameError` is raised if we mention a name (eg the name of a variable, function or module) that doesn't exist. This can happen if we mis-spell it, or forget to import.

In [2]:
sin(3.0)

NameError: name 'sin' is not defined

In [3]:
my_bool = true

NameError: name 'true' is not defined

`IndexError` is raised if we try to access a non-existent index in a `list` (or `tuple`, etc.)

In [4]:
L = [4, 5, 6]
L[19]

IndexError: list index out of range

`KeyError` is raised if we try to access a non-existent key in a `dict`:

In [5]:
d = {'name': 'Bob', 'age': 37}
d['address']

KeyError: 'address'

`ZeroDivisionError` is raised if we divide by zero:

In [6]:
10 / 0

ZeroDivisionError: division by zero

In [7]:
10 % 0

ZeroDivisionError: integer division or modulo by zero

The square root of a negative number is not defined, and we would see a `ValueError` with a useful extra piece of information: we tried a maths function with an argument in a **domain** where it was undefined.

In [8]:
from math import sqrt
sqrt(-3)

ValueError: math domain error

`TypeError` is raised in a few situations, eg trying to mutate a `tuple`:

In [9]:
t = (4, 5, 6)
t[0] = 17

TypeError: 'tuple' object does not support item assignment

`TypeError` is also raised if we try to index a `set` - remember, `set`s don't have order.

In [10]:
s = {4, 5, 6}
s[0]

TypeError: 'set' object is not subscriptable

```python
# don't run this - you might need to interrupt the kernel
x = ['aefiaejfaiefjaf'] * 100000000000
```

`MemoryError`

`FileNotFoundError` is one of a few operating system-related exceptions:

In [11]:
f = open('a_file_that_does_not_exist.txt')

FileNotFoundError: [Errno 2] No such file or directory: 'a_file_that_does_not_exist.txt'

### Tracebacks

In all of these cases, we saw a **Traceback**, which shows the sequence of function calls that led to the problem. As it says, **most recent call last**.  

In [12]:
def f(x): return x+1
def g(x): return f(x)*2
def h(x): return g(x)-1
def j(x): return 1/h(x)

In [13]:
j(0.5)

0.5

In [14]:
j(-0.5)

ZeroDivisionError: float division by zero

Tracebacks are there to help you debug your code!

### Handling exceptions with `try`-`except`

Normally, if an exception occurs, the program crashes! (And we see the traceback.)


<center><img src=img/exception_py.png width=40%></center>

What if we don't want to crash - we want to handle the exception somehow?

Eg, the people who made Jupyter Notebook (in Python) don't want to **crash** - they just want to show the error and let the user continue somehow.

In Python the exception-handling mechanism is called `try`-`except`. (Some other languages have the same mechanism with the name `try`-`catch`.)

`try`-`except` is a **control-flow** mechanism, analogous to `if`-`elif`-`else`. It
means something like: run **this** (the `try` block), and if there's an exception, clean up
by running **something else** (the `except` block) instead.

In [15]:
def f(d, k, x):
    try:
        print(x / d[k])
    except KeyError:
        print(f"The key {k} doesn't exist in the input {d}")

In [16]:
d = {'a': 0, 'b': 2}
f(d, 'c', 10)

The key c doesn't exist in the input {'a': 0, 'b': 2}


We can have multiple `except` blocks, and multiple types of errors per-block, and even a default "catch anything" block:

In [17]:
def f(d, k, x):
    try:
        print(x / d[k])
    except KeyError:
         print(f"The key {k} doesn't exist in the input {d}")
    except (ZeroDivisionError, FloatingPointError):
        print("You tried to divide by zero or to do something numerically bad")
    except:
        print("Something else bad happened")

In [18]:
d = {'a': 0, 'b': 2}
f(d, 'a', 10)

You tried to divide by zero or to do something numerically bad


Raising exceptions with `raise`
---

So far, we've seen how to handle ("catch") exceptions. What about
raising ("throwing") them? The most common scenario is when our
function receives arguments which are "impossible" in some way, or
when processing data that doesn't make sense. It's common to `raise ValueError` in that case.

**Heron's formula** is a formula for the area of a triangle:

$A = \sqrt{s (s-x) (s-y) (s-z)}$

where

$s = (x+y+z)/2$

In [20]:
def herons_formula(x, y, z):
    # Heron's formula [https://www.mathsisfun.com/geometry/herons-formula.html] 
    s = (x + y + z)/2.0
    area = sqrt(s * (s-x) * (s-y) * (s-z))
    return area

In [21]:
herons_formula(3, 4, 5) # right-angled => easy to calculate in our heads

6.0

But anytime we see an operation like square root, we should be on the alert. The square root of a negative number is not defined!


Could this happen in Heron's formula? Suppose we have values $x=2, y=2, z=10$. Then $s=7$, $s-z=-3$, and we'll try to take a square root of a negative number.

In [22]:
herons_formula(2, 2, 10)

ValueError: math domain error

But of course, with $x=2, y=2, z=10$, it is not a valid triangle! No such triangle exists. The $x$ and $y$ sides are too short to meet. So, the real issue here is the user gave us **bad values**. It's now good practice for us to check the values in the function, and `raise` an informative `ValueError` to help the user:

In [23]:
def herons_formula(x, y, z):
    """Calculate the area of a triangle (not necessarily right-angled) 
    given its three sides. Heron's formula tells us the area. However, 
    not all values of x, y, and z are valid. If the two smallest of 
    them add up to less than the largest, no such triangle exists.
    """
    # Order so that x <= y <= z
    x, y, z = sorted((x, y, z))
    # Check that the values are valid
    if x + y < z:
        # ... and if not, raise an exception.
        raise ValueError(f"No such triangle exists with sides {x}, {y}, and {z}.")

    # Heron's formula [https://www.mathsisfun.com/geometry/herons-formula.html] 
    s = (x + y + z)/2.0
    area = sqrt(s * (s-x) * (s-y) * (s-z))
    return area

In [24]:
herons_formula(3, 4, 5)

6.0

In [25]:
herons_formula(10, 2, 2) # we detect an impossible triangle and raise an exception

ValueError: No such triangle exists with sides 2, 2, and 10.

### Exceptions in doctests

We can take account of
exceptions in our *doctests*. That is, we can write a doctest where the
expected result is a Traceback and exception. For example, we can
write a doctest like the following. Note, we can put `...` to indicate some
extra text that the doctest can ignore.


In [26]:
def herons_formula(x, y, z):
    """
    A normal, working call:
    >>> herons_formula(3, 4, 5) # right-angled => easy to calculate in our heads
    6.0

    >>> herons_formula(10, 2, 2) # impossible => test that our ValueError is raised
    Traceback (most recent call last):
    ...
    ValueError: No such triangle exists
    """
    # Order so that x <= y <= z
    x, y, z = sorted((x, y, z))
    # Check that the values are valid
    if x + y < z:
        # ... and if not, raise an exception.
        raise ValueError("No such triangle exists")

    # Heron's formula [https://www.mathsisfun.com/geometry/herons-formula.html] 
    s = (x + y + z)/2.0
    area = sqrt(s * (s-x) * (s-y) * (s-z))
    return area   

In [27]:
import doctest
doctest.testmod()

TestResults(failed=0, attempted=2)